В этом посте будет рассмотрено то, как оптимизировать портфель путем изменения весов входящих в него активов. Под оптимизацией портфеля понимантся такое соотношение весов, которое будет уловлетворять одному из условий:

* Портфель с минимальным уровнем риском при желаемой доходности;
* Портфель с максимальной доходностью при установленном риске;
* Портфель с максимальным значением доходности

В данном посте будут рассмотрены девять акций, которые предлагает торговый робот одного из брокеров и в конце будет рассмотрено - совпадают ли веса. Так как имеется значение показателя на начало января - можно будет приблизительно оценить как сильно отличается доходность и риски робота от выполненного ручного (т.е. при помощи кода на python) расчета. 

После недолгого анализа робот мне предложил сформировать портфель из девяти акций: 'ATVI','BA','CNP','CMA', 'STZ','GPN','MPC','NEM' и 'PKI'. Так как рассматриваем инвестирование, а не трейдинг - будем рассматривать временной горизонт три года. Соответственно и данне возьмем за этот период.

In [1]:
#Загружаю библиотеки

import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ticker = ['ATVI','BA','CNP','CMA', 'STZ','GPN','MPC','NEM', 'PKI']

In [25]:
stock = yf.download(ticker,'2017-01-01', '2019-01-31')

[*********************100%***********************]  9 of 9 downloaded


Веса всех акций, входящих в портфель должны составлять единицу. В начале будут взяты те, которые предложил робот. Будет расчитана доходность и стандартное отклонение портфеля. Затем будет использована  симуляция Монте-Карло для получения оптимальных весов. 

**Симуляция Монте-Карло**

Приведу краткое описание, которое приводиться в одном из постов - как применяется метод Монте -Карлоя для подбора оптимального портфеля. 

Сначала акциям задаются случайные веса. Производиться расчет доходности и стандартного отклонения и сохраняем его. Потом случайным образом меняются веса (главное не забывать, что их сумма должна составлять единицу)и повторяется предыидущих шаг - расчет и сохранение полученного значения. Количество итераций зависит от времени, машинки для расчета и риска, который готов принять инвестор. В этом посте попробуем провести 10000 расчетов для выявления портфеля, который будет удовлетворять любому из трех условий.

**Оптимизация портфеля на Python**

In [26]:
# Выделение скорректированой цены закрытия
all_adj_close = stock[['Adj Close']]
all_adj_close.round(2).head()

Adj Close                                                          
                ATVI      BA    CMA    CNP    GPN    MPC    NEM    PKI     STZ
Date                                                                          
2017-01-03     35.94  145.53  62.92  21.58  70.27  47.81  32.28  52.30  148.24
2017-01-04     36.65  147.06  63.45  21.77  72.89  46.28  32.59  52.61  151.34
2017-01-05     37.22  147.15  63.45  21.77  73.18  46.11  34.09  52.04  140.58
2017-01-06     37.19  147.51  64.12  22.26  74.21  45.50  33.02  52.42  143.15
2017-01-09     36.98  146.79  63.45  21.83  79.53  44.67  32.96  52.85  143.95

Дальше необходимо получить ежедневную доходность по каждой акции

In [28]:
all_returns = all_adj_close.pct_change()

In [29]:
all_returns.round(4).head()*100

Adj Close                                                
                ATVI    BA   CMA   CNP   GPN   MPC   NEM   PKI   STZ
Date                                                                
2017-01-03       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2017-01-04      1.98  1.05  0.84  0.88  3.73 -3.20  0.96  0.59  2.09
2017-01-05      1.56  0.06  0.00  0.00  0.40 -0.37  4.60 -1.08 -7.11
2017-01-06     -0.08  0.24  1.06  2.25  1.41 -1.32 -3.14  0.73  1.83
2017-01-09     -0.56 -0.49 -1.04 -1.93  7.17 -1.82 -0.18  0.82  0.56

Теперь можно вычислить доходность каждой акции и построить ковариационную матрицу для того чтобы оценить силу взаимосвязи элементов. 

In [35]:
mean_returns = all_returns.mean()
cov_matrix = all_returns.cov()
print (mean_returns)
print (cov_matrix)

Adj Close  ATVI    0.000712
           BA      0.001957
           CMA     0.000484
           CNP     0.000595
           GPN     0.001025
           MPC     0.000647
           NEM     0.000140
           PKI     0.001144
           STZ     0.000334
dtype: float64
               Adj Close                                                    \
                    ATVI        BA       CMA       CNP       GPN       MPC   
Adj Close ATVI  0.000485  0.000135  0.000071  0.000029  0.000148  0.000110   
          BA    0.000135  0.000268  0.000091  0.000027  0.000105  0.000109   
          CMA   0.000071  0.000091  0.000216  0.000009  0.000072  0.000092   
          CNP   0.000029  0.000027  0.000009  0.000099  0.000021  0.000019   
          GPN   0.000148  0.000105  0.000072  0.000021  0.000217  0.000084   
          MPC   0.000110  0.000109  0.000092  0.000019  0.000084  0.000282   
          NEM   0.000013  0.000022 -0.000016  0.000012  0.000011  0.000032   
          PKI   0.000132  0.000